In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Libraries imported.


In [16]:
CLIENT_ID = 'IKVJZO3L2RKHD12H35LD3S40Z51JWXA4NN4XL3PH00GK2QRU' # your Foursquare ID
CLIENT_SECRET = 'FAMQK0S5IWNNSM00JNCZXYXLK4XUBIMSPY3JSWCFG0XE0XVP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index = True)

<ipython-input-7-b3a0b7e7a92f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-7-b3a0b7e7a92f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-7-b3a0b7e7a92f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-7-b3a0b7e7a92f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-7-b3a0b7e7a92f>:9: FutureWarning: The frame.append method is deprecated and w

In [8]:
neighborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [11]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [12]:
brooklyn_data.drop(['Borough'], axis = 1, inplace = True)
brooklyn_data.loc[brooklyn_data['Neighborhood']== 'Williamsburg']

,Neighborhood,Latitude,Longitude
15,Williamsburg,40.707144,-73.958115


In [13]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [15]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
#function that gets the nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
LIMIT = 100
radius =500
brooklyn_venues = getNearbyVenues(names = brooklyn_data['Neighborhood'],
                                   latitudes = brooklyn_data['Latitude'],
                                   longitudes = brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [19]:
brooklyn_venues.groupby('Neighborhood').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,86,86,86,86,86,86
Bedford Stuyvesant,32,32,32,32,32,32
Bensonhurst,28,28,28,28,28,28
Bergen Beach,6,6,6,6,6,6
...,...,...,...,...,...,...
Vinegar Hill,30,30,30,30,30,30
Weeksville,15,15,15,15,15,15
Williamsburg,30,30,30,30,30,30


In [20]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Pc_venues_sorted = pd.DataFrame(columns=columns)
Pc_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    Pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

In [23]:
Pc_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Bubble Tea Shop,Gas Station,Pizza Place,Fast Food Restaurant,Italian Restaurant,Pharmacy,Sushi Restaurant,Bus Stop,Deli / Bodega
1,Bay Ridge,Pizza Place,Italian Restaurant,Spa,Bagel Shop,Greek Restaurant,Bar,American Restaurant,Grocery Store,Chinese Restaurant,Middle Eastern Restaurant
2,Bedford Stuyvesant,Bar,Deli / Bodega,Coffee Shop,Italian Restaurant,Boutique,Tiki Bar,Park,Community Center,Cocktail Bar,Discount Store
3,Bensonhurst,Italian Restaurant,Donut Shop,Sushi Restaurant,Chinese Restaurant,Ice Cream Shop,Bakery,Bar,Spa,Cha Chaan Teng,Shabu-Shabu Restaurant
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Park,Baseball Field,Playground,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery


In [24]:
kclusters = 5
brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)
#add cluster labels
Pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
brooklyn_merged = brooklyn_data
brooklyn_merged = brooklyn_merged.join(Pc_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
brooklyn_merged.head() 

<ipython-input-24-fe521db03f3f>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,40.625801,-74.030621,0,Pizza Place,Italian Restaurant,Spa,Bagel Shop,Greek Restaurant,Bar,American Restaurant,Grocery Store,Chinese Restaurant,Middle Eastern Restaurant
1,Bensonhurst,40.611009,-73.995180,0,Italian Restaurant,Donut Shop,Sushi Restaurant,Chinese Restaurant,Ice Cream Shop,Bakery,Bar,Spa,Cha Chaan Teng,Shabu-Shabu Restaurant
2,Sunset Park,40.645103,-74.010316,0,Bank,Bakery,Mexican Restaurant,Latin American Restaurant,Grocery Store,Gym,Fried Chicken Joint,Pizza Place,Pharmacy,Deli / Bodega
3,Greenpoint,40.730201,-73.954241,0,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Yoga Studio,Restaurant,French Restaurant,Deli / Bodega,Café,Tea Room
4,Gravesend,40.595260,-73.973471,0,Pizza Place,Italian Restaurant,Chinese Restaurant,Deli / Bodega,Bus Station,Bakery,Jewelry Store,Bar,Pharmacy,Spa


In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, nei, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
brooklyn_merged.loc[brooklyn_merged['Neighborhood'] == 'Williamsburg']


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Williamsburg,40.707144,-73.958115,0,Pizza Place,Bagel Shop,Yoga Studio,Clothing Store,Tapas Restaurant,Grocery Store,Gym,Park,Coffee Shop,Taco Place


In [27]:
cluster3 = brooklyn_merged[brooklyn_merged['Cluster Labels'] == 0 ].reset_index(drop = True)
cluster3.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,40.625801,-74.030621,0,Pizza Place,Italian Restaurant,Spa,Bagel Shop,Greek Restaurant,Bar,American Restaurant,Grocery Store,Chinese Restaurant,Middle Eastern Restaurant
1,Bensonhurst,40.611009,-73.995180,0,Italian Restaurant,Donut Shop,Sushi Restaurant,Chinese Restaurant,Ice Cream Shop,Bakery,Bar,Spa,Cha Chaan Teng,Shabu-Shabu Restaurant
2,Sunset Park,40.645103,-74.010316,0,Bank,Bakery,Mexican Restaurant,Latin American Restaurant,Grocery Store,Gym,Fried Chicken Joint,Pizza Place,Pharmacy,Deli / Bodega
3,Greenpoint,40.730201,-73.954241,0,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Yoga Studio,Restaurant,French Restaurant,Deli / Bodega,Café,Tea Room
4,Gravesend,40.595260,-73.973471,0,Pizza Place,Italian Restaurant,Chinese Restaurant,Deli / Bodega,Bus Station,Bakery,Jewelry Store,Bar,Pharmacy,Spa


In [28]:
cl3_coffee = cluster3.loc[cluster3['1st Most Common Venue'] == 'Coffee Shop'].reset_index(drop = True)
cl3_coffee

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Crown Heights,40.670829,-73.943291,0,Coffee Shop,Café,Museum,Pizza Place,Playground,Supermarket,Candy Store,Bookstore,Cosmetics Shop,Bank
1,Cobble Hill,40.687920,-73.998561,0,Coffee Shop,Cocktail Bar,Playground,Bakery,Bar,Italian Restaurant,Pizza Place,Deli / Bodega,Dive Bar,Flower Shop
2,Park Slope,40.672321,-73.977050,0,Coffee Shop,Pizza Place,Burger Joint,Bagel Shop,Bakery,Pet Store,Bookstore,Pub,Spa,Bubble Tea Shop
3,Downtown,40.690844,-73.983463,0,Coffee Shop,Burger Joint,Chinese Restaurant,Hotel,Grocery Store,Performing Arts Venue,Sandwich Place,Arts & Crafts Store,Spa,Bar
4,Boerum Hill,40.685683,-73.983748,0,Coffee Shop,Bar,Hotel,Dance Studio,Spa,Furniture / Home Store,Arts & Crafts Store,Yoga Studio,Cosmetics Shop,Sandwich Place
5,North Side,40.714823,-73.958809,0,Coffee Shop,Pizza Place,Yoga Studio,Bakery,Wine Bar,Bar,Dive Bar,Cocktail Bar,Thai Restaurant,Restaurant
6,Vinegar Hill,40.703321,-73.981116,0,Coffee Shop,Food Truck,Bike Rental / Bike Share,Art Gallery,Café,Garden,Factory,Performing Arts Venue,Gym / Fitness Center,Pharmacy
7,Dumbo,40.703176,-73.988753,0,Coffee Shop,Park,Scenic Lookout,Ice Cream Shop,Bookstore,Boxing Gym,Bakery,Pizza Place,Italian Restaurant,Yoga Studio


In [29]:
map_coffee = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, label in zip(cl3_coffee['Latitude'], cl3_coffee['Longitude'], cl3_coffee['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_coffee)  
    
map_coffee